# Random Histogram Forest

In [1]:
from timeit import timeit

### Testing

In [2]:

setup_code='''
import scipy.io as sio
import scipy.stats as sstats
import random 
import numpy as np
import rht
import rhf
import anomaly_score as a_s
import Node
# set the number of trees and max height
H = 5
T = 100
mat_contents = sio.loadmat("../datasets/satellite.mat")
dataset = mat_contents['X'] 
labels = mat_contents['y']
dataset = dataset.astype('float32') 
'''
code = '''
test_rhf = rhf.rhf(X=dataset, t=T, nd=0, h=H)
for i, x in enumerate(dataset):
    score = a_s.anomaly_score(test_rhf, dataset.size, x)
'''
for i in range(0,10):
    print("Total time for rhf-cython (train) = ", timeit(setup=setup_code, stmt=code, number=1))

'''
for i, x in enumerate(dataset):
    score = anomaly_score(test_rhf, dataset.size, x)
    np.append(scores, [score, labels[i]])
    print("score = " + str(score) + " | anomaly? " + str(labels[i]))
'''
    
    


Total time for rhf-cython (train) =  276.1681841409991
Total time for rhf-cython (train) =  276.6936287569988
Total time for rhf-cython (train) =  260.6195142760007
Total time for rhf-cython (train) =  253.41527432000294
Total time for rhf-cython (train) =  246.1381310709985
Total time for rhf-cython (train) =  243.79540850400008
Total time for rhf-cython (train) =  238.01656887500212
Total time for rhf-cython (train) =  241.9002185399986
Total time for rhf-cython (train) =  238.15488693099906
Total time for rhf-cython (train) =  251.28006793000168


'\nfor i, x in enumerate(dataset):\n    score = anomaly_score(test_rhf, dataset.size, x)\n    np.append(scores, [score, labels[i]])\n    print("score = " + str(score) + " | anomaly? " + str(labels[i]))\n'

### Plotting

In [ ]:
()